In [7]:
import json
import re
import PyPDF2

def extract_info_to_dict(file_path):
    """
    Extracts specific information from a document and saves it into a dictionary.

    Args:
        file_path (str): The path to the document file.

    Returns:
        dict: A dictionary containing the extracted information.
              Returns None if the file cannot be processed.
    """

    bid_fields = {
        "Avis n°": None,
        "nom officiel": None,
        "Nom complet de l'acheteur": None,
        "Type de Numéro national d'identification": None,
        "N° National d'identification": None,
        "Ville": None,
        "Code postal": None,
        "Groupement de commandes": None,
        "Nom du contact": None,
        "Adresse mail du contact": None,
        "Date et heure limite de réception des plis": None,
        "Possibilité d'attribution sans négociation": None,
        "Description succincte du marché": None,
        "Lieu principal d'exécution du marché": None,
        "Durée du marché (en mois)": None,
        "Critères de sélection (type, Description, poids)": []
    }
    text = ""
    try:
        with open(file_path, 'rb') as pdf_file:  # Open in binary read mode ('rb')
            reader = PyPDF2.PdfReader(pdf_file)
            print(f"Document {file_path} avec {len(reader.pages)} pages")
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text() or ""  # Extract text, handle None
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

    # Extraction patterns (more robust)
    patterns = {
        "Avis num": r"Identifiant    interne: ([A-Za-z0-9/]+)",
        "nom officiel": r"Nom officiel: ([^\n]+)",
        "Nom complet acheteur": r"Nom complet de l'acheteur: ([^\n]+)",
        "Type de Numéro national d'identification": r"Type de Numéro national d'indentification: ([^\n]+)",
        "Num National d'identification": r"Nº National d'identification: ([^\n]+)",
        "Ville": r"Ville: ([^\n]+)",
        "Code postal": r"Code postal: ([^\n]+)",
        "Groupement de commandes": r"Groupement de commandes: ([^\n]+)",
        "Nom du contact": r"Nom du contact: ([^\n]+)",
        "Adresse mail du contact": r"Adresse électronique: ([^\n]+)",
        "Date et heure limite de réception des plis": r"Date limite de réception des offres: ([^\n]+)",
        "Possibilité d'attribution sans négociation": r"Possibilité d'attribution sans négociation: ([^\n]+)",
        "Description succincte du marché": r"Description: ([^\n]+)",
        "Lieu principal d'exécution du marché": r"Adresse postale: ([^\n]+)",
        "Durée du marché (en mois)": r"Durée: (\d+) Mois",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            bid_fields[key] = match.group(1).strip()

    # Extracting Critères de sélection (type, Description, poids) - Adapt to capture multiple
    criteria_matches = re.finditer(r"Critère:\s*Type:\s*([^\n]+)\s*Description:\s*([^\n]+)", text)
    for match in criteria_matches:
        bid_fields["Critères de sélection (type, Description, poids)"].append({
            "type": match.group(1).strip(),
            "Description": match.group(2).strip(),
            "poids": None  # Poids is consistently null in your example
        })

    return bid_fields

# Example Usage
#file_name = r"C:\Users\jch_m\Documents\DDM\AO-2518-27052025-05-02_14-27-391.pdf" 
file_name = r"C:\Users\jch_m\Documents\DDM\4169978.pdf"
extracted_data = extract_info_to_dict(file_name)

if extracted_data:
    print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
else:
    print("No data extracted.")



Document C:\Users\jch_m\Documents\DDM\4169978.pdf avec 4 pages
{
    "Avis n°": null,
    "nom officiel": null,
    "Nom complet de l'acheteur": null,
    "Type de Numéro national d'identification": null,
    "N° National d'identification": null,
    "Ville": null,
    "Code postal": null,
    "Groupement de commandes": null,
    "Nom du contact": null,
    "Adresse mail du contact": null,
    "Date et heure limite de réception des plis": null,
    "Possibilité d'attribution sans négociation": null,
    "Description succincte du marché": null,
    "Lieu principal d'exécution du marché": null,
    "Durée du marché (en mois)": null,
    "Critères de sélection (type, Description, poids)": []
}
